# Crypto Price Data Import & Resample

#### Download Links:
- [Kaggle Minutely Cryptocurrency Price History](https://www.kaggle.com/tencars/392-crypto-currency-pairs-at-minute-resolution)
- [Get Historical Crypto Price Data YouTube Video](https://youtu.be/dCBLFPlZcck)

## 1. Get Imports

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

## 2. Import Universe

In [ ]:
from zipfile import ZipFile
zf = ZipFile('/home/leosmigel/Downloads/archive.zip')
cols = ['time', 'open', 'high', 'low', 'close', 'volume']

In [ ]:
dfs = pd.concat({text_file.filename.split('.')[0]: pd.read_csv(zf.open(text_file.filename),
                                                              usecols=cols)
                
                for text_file in zf.infolist()
                if text_file.filename.endswith('.csv')
                })
dfs

In [ ]:
df = dfs.droplevel(1).reset_index().rename(columns={'index':'ticker'})
df = df[df['ticker'].str.contains('usd')]
df['date'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values(by=['date','ticker'])
df = df.drop(columns='time')
df = df.set_index(['date','ticker'])
df = df['2020-07-01':'2020-12-31']
df

## 4. Resample Timeframes

In [ ]:
bars1m = df
bars1m = bars1m.reset_index().set_index('date').groupby('ticker').resample('1min').last().droplevel(0)
bars1m.loc[:, bars1m.columns[:-1]] = bars1m[bars1m.columns[:-1]].ffill()
bars1m.loc[:, 'volume'] = bars1m['volume'].fillna(value=0.0)
bars1m = bars1m.reset_index().sort_values(by=['date','ticker']).set_index(['date','ticker'])
bars1m

# Insert Price Data

## 1. Get Symbols

In [ ]:
tickers = bars1m.index.get_level_values(1).unique()
tickers

In [ ]:
active_tickers = bars1m['2020-12-30':].index.get_level_values(1).unique()
active_tickers

## 2. Create Symbols Dataframe

In [ ]:
symbols = pd.DataFrame(tickers)
symbols['name'] = symbols['ticker']
symbols['market'] = 'crypto'
symbols['active'] = np.where(symbols['ticker'].isin(active_tickers), True, False)
symbols = symbols.sort_values(by='ticker')
symbols

## 3. Create MinuteBar DataFrame

In [ ]:
minute_bars = bars1m.reset_index().sort_values(by=['ticker','date']).set_index(['ticker','date'])
minute_bars

## 4. Insert Data

In [ ]:
from psql import db, session
from models import Symbol, MinuteBar

In [ ]:
for i, r in symbols.iterrows():
    symbol = Symbol(ticker=r['ticker'],
                    name=r['name'],
                    market=r['market'],
                    active=r['active'])
    session.add(symbol)
    session.commit()
    
    bars = minute_bars.xs(r['ticker']).reset_index()
    bars['symbol_id'] = symbol.id
    
    session.bulk_insert_mappings(MinuteBar,
                                bars.to_dict(orient='records'))
    session.commit()